In [1]:
import pandas as pd
# Считывание данных
hotels = pd.read_csv(r'D:\IDE_github\EDA\EDA_project\data\hotels.csv')
# Вывод первых нескольких строк
hotels.head()

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643
3,216 Avenue Jean Jaures 19th arr 75019 Paris Fr...,34,9/22/2015,7.5,Mercure Paris 19 Philharmonie La Villette,United Kingdom,No Negative,0,607,Friendly staff quiet comfortable room spotles...,11,8,10.0,"[' Leisure trip ', ' Solo traveler ', ' Standa...",681 day,48.888697,2.394540
4,Molenwerf 1 1014 AG Amsterdam Netherlands,914,3/5/2016,8.5,Golden Tulip Amsterdam West,Poland,Torn sheets,4,7586,The staff was very friendly and helpful Break...,20,10,9.6,"[' Business trip ', ' Couple ', ' Standard Dou...",516 day,52.385601,4.847060


In [2]:
# Создание копии датасета
data = hotels.copy()
# Вывод информации о пропусках
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

In [3]:
# Вывод типа данных каждого столбца
data.dtypes

hotel_address                                  object
additional_number_of_scoring                    int64
review_date                                    object
average_score                                 float64
hotel_name                                     object
reviewer_nationality                           object
negative_review                                object
review_total_negative_word_counts               int64
total_number_of_reviews                         int64
positive_review                                object
review_total_positive_word_counts               int64
total_number_of_reviews_reviewer_has_given      int64
reviewer_score                                float64
tags                                           object
days_since_review                              object
lat                                           float64
lng                                           float64
dtype: object

In [4]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х — данные с информацией об отелях, у — целевая переменная (рейтинги отелей) 
X = data.drop(['reviewer_score'], axis = 1)  
y = data['reviewer_score']
# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [5]:
# Импортируем необходимые библиотеки:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  
# Создаём модель  
regr = RandomForestRegressor(n_estimators=100)
# Учет только численных данных
X_train = X_train.select_dtypes(include=['int64', 'float64'])
X_test = X_test.select_dtypes(include=['int64', 'float64'])
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train) 
# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)

In [6]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются  
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.  
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

MAPE: 0.1414300865124694


Далее стоит задача повышения точности прогнозов. Нужно подготовить данные

## 1. Работа со строковыми значениями

In [7]:
# Вывод столбцов, содержащих строковые значения
data_columns_obj = data.select_dtypes(include=['object']).columns.to_list()
data_columns_obj 

['hotel_address',
 'review_date',
 'hotel_name',
 'reviewer_nationality',
 'negative_review',
 'positive_review',
 'tags',
 'days_since_review']

### 1.1 Адрес отеля

In [8]:
data['hotel_address'].head(2)

0    Stratton Street Mayfair Westminster Borough Lo...
1    130 134 Southampton Row Camden London WC1B 5AF...
Name: hotel_address, dtype: object

In [9]:
data['hotel_address'].value_counts()

hotel_address
163 Marsh Wall Docklands Tower Hamlets London E14 9SJ United Kingdom              3587
372 Strand Westminster Borough London WC2R 0JJ United Kingdom                     3206
Westminster Bridge Road Lambeth London SE1 7UT United Kingdom                     3095
Scarsdale Place Kensington Kensington and Chelsea London W8 5SY United Kingdom    2688
7 Pepys Street City of London London EC3N 4AF United Kingdom                      2379
                                                                                  ... 
Via Palmanova 153 20132 Milan Italy                                                  7
40 Rue Rene Boulanger 10th arr 75010 Paris France                                    7
Via M Buonarroti 13 Fiera Milano City 20149 Milan Italy                              6
Johann Staud Stra e 32 16 Ottakring 1160 Vienna Austria                              6
13 Rue Fran ois Ory 92120 Paris France                                               5
Name: count, Length: 1493, dt

Столбцы содержат название города и страны расположения отеля, их можно излвечь с помощью регулярных выражений

In [10]:
# Импорт бибилиотеки для работы с регулрными выражениями
import re
# Регулярное выражение для поиска сочетаний слов в конце строки
regular_form = r'([A-Z][a-z]+(?: [A-Z][a-z]+)*)$'
data['city_country'] = data['hotel_address'].apply(lambda x: re.search(regular_form, x).group())

In [11]:
# Вычленияем страну
data['country'] = data['city_country'].apply(lambda x: x if x == 'United Kingdom' else x.split()[1])
# Поиск города
data['city'] = data['city_country'].apply(lambda x: 'London' if x == 'United Kingdom' else x.split()[0])

In [12]:
# Удаление старого признака
data = data.drop(['hotel_address'], axis=1)

### 1.2 Дата

In [13]:
data['review_date'].head(2)

0    2/19/2016
1    1/12/2017
Name: review_date, dtype: object

Можно использовать для извлечения дня, месяца или года

In [14]:
# Пребобразование в формат datetime - удобно извлекать
data['review_date'] = pd.to_datetime(data['review_date'])

### 1.3 Название отеля

In [15]:
data['hotel_name'].head(2)

0                 The May Fair Hotel
1    Mercure London Bloomsbury Hotel
Name: hotel_name, dtype: object

Можно использовать как наблюдение за самым распространенным в списке отелем. Но для модели создавать численный признак из этого затратно\
Поэтому из названия предлагается вытащить принадлежность отеля к какой-либо категории

In [16]:
data['hotel_name'].value_counts()

hotel_name
Britannia International Hotel Canary Wharf           3587
Strand Palace Hotel                                  3206
Park Plaza Westminster Bridge London                 3095
Copthorne Tara Hotel London Kensington               2688
DoubleTree by Hilton Hotel London Tower of London    2379
                                                     ... 
Ibis Styles Milano Palmanova                            7
Renaissance Paris Republique Hotel Spa                  7
Hotel Wagner                                            6
Hotel Gallitzinberg                                     6
Mercure Paris Porte d Orleans                           5
Name: count, Length: 1492, dtype: int64

In [ ]:
# Расширенный список брендов с учетом вариаций
brand_pattern = r'(?i)\b(Hilton|Marriott|Ibis|Mercure|Renaissance|DoubleTree|Britannia|Park Plaza|Copthorne|Holiday Inn|Best Western|Hyatt|Radisson|Sheraton|Novotel|InterContinental|Crowne Plaza|Ramada|Wyndham|Four Seasons|Ritz-Carlton|Shangri-La|Sofitel|Westin|W Hotels|St\.? Regis|Fairmont|Swissotel|Pullman|Grand Hyatt|Kimpton|Mövenpick|Millennium|Thistle|Travelodge|Premier Inn|Motel One|B&B)\b'
# Извлечение с учетом регистра и частичных совпадений
data['brand'] = data['hotel_name'].str.extract(brand_pattern, flags=re.IGNORECASE)[0]
# Замена похожих названий
brand_mapping = {
    'Holiday Inn Express': 'Holiday Inn',
    'Courtyard by Marriott': 'Marriott',
    'JW Marriott': 'Marriott',
    'Marriott Hotels': 'Marriott'
}
# Статистика по брендам
brand_stats = data['brand'].value_counts()

In [ ]:
brand_stats

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 20 columns):
 #   Column                                      Non-Null Count   Dtype         
---  ------                                      --------------   -----         
 0   additional_number_of_scoring                386803 non-null  int64         
 1   review_date                                 386803 non-null  datetime64[ns]
 2   average_score                               386803 non-null  float64       
 3   hotel_name                                  386803 non-null  object        
 4   reviewer_nationality                        386803 non-null  object        
 5   negative_review                             386803 non-null  object        
 6   review_total_negative_word_counts           386803 non-null  int64         
 7   total_number_of_reviews                     386803 non-null  int64         
 8   positive_review                             386803 non-null  object       

In [30]:
# Создание списков слов-индикаторов для оценки уровня отеля
luxury_keywords = ['Plaza', 'Palace', 'Royal', 'Boutique']
business_keywords = ['Hilton', 'Marriott', 'DoubleTree']
budget_keywords = ['Ibis', 'Travelodge']
# Классификация
data['class'] = 'other'
data.loc[data['hotel_name'].str.contains('|'.join(luxury_keywords)), 'class'] = 'luxury'
data.loc[data['hotel_name'].str.contains('|'.join(business_keywords)), 'class'] = 'business'
data.loc[data['hotel_name'].str.contains('|'.join(budget_keywords)), 'class'] = 'budget'

In [33]:
# Наличие терминов Spa
data['has_spa'] = data['hotel_name'].str.contains('Spa').astype(int)
# Количество слов
data['word_count'] = data['hotel_name'].str.split().str.len()
# Словарь типов с учетом синонимов и международных вариантов
type_patterns = {
    'Hotel': r'(?i)\b(Hotel|Hotels|Hotellerie|Hoteles|Hôtel|Hôtels)\b',
    'Resort': r'(?i)\b(Resort|Resorts|Complex|Beach Club|Retreat)\b',
    'Boutique': r'(?i)\b(Boutique|Design Hotel|Chic Hotel|Luxury Lofts)\b',
    'Inn': r'(?i)\b(Inn|Gasthof|Pension|Guesthouse|Bed & Breakfast|B&B)\b',
    'Apartment': r'(?i)\b(Apartment|Apartments|Aparthotel|Residence|Suites|Studio|Loft)\b',
    'Hostel': r'(?i)\b(Hostel|Backpackers|Youth Hostel|Dormitory)\b',
    'Villa': r'(?i)\b(Villa|Villas|Château|Manor|Mansion|Cottage)\b',
    'Lodge': r'(?i)\b(Lodge|Cabins|Chalet|Bungalow|Ecolodge)\b',
    'Motel': r'(?i)\b(Motel|Motor Inn|Roadhouse)\b',
    'Palace': r'(?i)\b(Palace|Grand Hotel|Luxury Hotel)\b',
    'Ryokan': r'(?i)\b(Ryokan|Onsen|Minshuku)\b',
    'Capsule': r'(?i)\b(Capsule|Pod Hotel)\b'
}
# Функция для извлечения типа с приоритетом более специфичных совпадений
def extract_hotel_type(name):
    name = str(name)
    matches = []
    for type_name, pattern in type_patterns.items():
        if re.search(pattern, name):
            # Приоритет по длине совпадения (более длинные = более специфичные)
            match = re.search(pattern, name).group()
            matches.append((len(match), type_name))
    if matches:
        # Возвращаем самый специфичный тип
        return sorted(matches, reverse=True)[0][1]
    return 'Other'  # Для неклассифицированных вариантов
# Применяем функцию
data['type'] = data['hotel_name'].apply(extract_hotel_type)

In [35]:
data['type'].value_counts()

type
Other        185327
Hotel        162571
Inn           13180
Palace        13098
Apartment      6055
Boutique       3102
Villa          1934
Lodge          1536
Name: count, dtype: int64

Other много. Имеет смысл заменить его на самый распространенный тип в городе

In [51]:
# Находим самый частый тип для каждого города
city_type_mapping = (
    data[data['type'] != 'Other']  # Исключаем 'Other' при расчете моды
    .groupby('city')['type']
    .agg(lambda x: x.mode()[0] if not x.mode().empty else 'Hotel')  # По умолчанию 'Hotel'
    .to_dict()
)

# Заменяем 'Other' на самый частый тип в городе
data['type'] = data.apply(
    lambda row: city_type_mapping.get(row['city'], 'Hotel') 
    if row['type'] == 'Other' 
    else row['type'],
    axis=1
)

In [36]:
# Удаление старого признака
data = data.drop(['hotel_name'], axis=1)

### 1.4 Национальность рецензента

In [37]:
data['reviewer_nationality'].head(2)

0     United Kingdom 
1     United Kingdom 
Name: reviewer_nationality, dtype: object

In [38]:
# Извлечение страны
data['reviewer_country'] = data['reviewer_nationality'].str.strip()
# Группировка по регионам (пример)
europe = ['United Kingdom', 'France', 'Germany', 'Spain', 'Italy']
asia = ['China', 'Japan', 'India', 'United Arab Emirates']
data['reviewer_region'] = data['reviewer_nationality'].apply(
    lambda x: 'Europe' if x in europe else ('Asia' if x in asia else 'Other')
)
# Гость из той же страны, что и отель
data['is_local'] = (data['reviewer_nationality'] == data['country']).astype(int)

In [39]:
# Удаление старого признака
data = data.drop(['reviewer_nationality'], axis=1)

### 1.5 Негативная оценка

In [40]:
data['negative_review'].head(2)

0           Leaving 
1     poor breakfast
Name: negative_review, dtype: object

In [41]:
# Словарь часто распространенных проблем
issues = {
    'cleanliness': ['dirty', 'clean', 'smell', 'stains'],
    'service': ['staff', 'rude', 'service', 'wait'],
    'amenities': ['WiFi', 'breakfast', 'bed', 'shower']
}
# Проверка на наличие проблемы из каждой категории в отзыве
for category, keywords in issues.items():
    data[f'issue_{category}'] = data['negative_review'].str.contains('|'.join(keywords), case=False).astype(int)

In [42]:
# Удаление старого признака
data = data.drop(['negative_review'], axis=1)

### 1.6 Положительная оценка

In [43]:
data['positive_review'].head(2)

0     Staff were amazing
1               location
Name: positive_review, dtype: object

In [44]:
# Словарь часто распространенных слов в позитивном отзыве
pos_categories = {
    'staff': ['friendly', 'helpful', 'polite', 'attentive'],
    'comfort': ['comfortable', 'cozy', 'spacious', 'quiet'],
    'cleanliness': ['clean', 'spotless', 'tidy'],
    'location': ['central', 'convenient', 'walkable'],
    'amenities': ['breakfast', 'pool', 'view', 'wifi']
}
# Проверка на наличие слов из каждой категории словаря с положительными сочетаниями в отзыве
for cat, keywords in pos_categories.items():
    data[f'pos_{cat}'] = data['positive_review'].str.contains('|'.join(keywords), case=False).astype(int)

In [45]:
# Удаление старого признака
data = data.drop(['positive_review'], axis=1)

### 1.7 Теги

In [46]:
data['tags'].head(2)

0    [' Leisure trip ', ' Couple ', ' Studio Suite ...
1    [' Business trip ', ' Couple ', ' Standard Dou...
Name: tags, dtype: object

Из тегов можно извлечь множество уникальных и выявить самый распространенный

In [47]:
# Множество тегов
unique_tags = {
    # Разделение строки по пробелу, удаление кавычек
    t.strip().strip("'") 
    for tag_str in data['tags']
    # Удаление скобок, разделение по запятой
    for t in tag_str.strip("[]").split(",")
}
print(unique_tags)

{' Hilton Queen Family Room ', ' Interconnecting Double Rooms ', ' Superior Room with Queensize Bed and double couch 2 adults 2 children ', ' Double or Twin Room with City View ', ' Duplex ', ' Classic Room ', ' Studio Disability Access ', ' One Bedroom Queen Suite ', ' Deluxe Queen Room with River View ', ' Superior Double Room with London Eye View ', ' Double or Twin Room with Parking ', ' Double Room ', ' Double Room with Extra Bed 2 Adults 1 Child with Ramblas View ', ' Deluxe Duplex Room ', ' Executive Double Room Top Floor ', ' Two Bedroom Deluxe Suite ', ' Superior Double Room with Terrace and Cathedral View ', ' Plaza Club Room ', ' Suite with Eiffel Tower View ', ' Ambassador Suite ', ' Deluxe Double Room with View and Disability Access ', ' Duplex King Suite 2 Adults 2 Children ', ' Double Studio ', ' King Room with Hyde Park View ', ' Four Seasons King Junior Suite ', ' Junior Suite with Private Access to Pool and Spa ', ' King Room with Eiffel Tower View ', ' Panorama Suite

In [48]:
from collections import Counter
# Создаем словарь для подсчета тегов
tag_counts = Counter()
for tag_str in data['tags']:
    # Преобразуем строку в список
    tags = [t.strip().strip("'") for t in tag_str.strip("[]").split(",")]
    # Обновляем счетчик
    tag_counts.update(tags)

# Находим самый частый тег
most_common_tag, count = tag_counts.most_common(1)[0]

print(f"Самый популярный тег: '{most_common_tag}' (встречается {count} раз(а))")

Самый популярный тег: ' Leisure trip ' (встречается 313593 раз(а))


Также можно получить признаки:
- тип поездки
- тип гостей
- категория номера
- длительность пребывания

In [52]:
# Список типов поездок с учетом возможных вариантов написания
trip_types = {
    'Leisure': ['Leisure trip', 'Vacation', 'Holiday', 'Leisure'],
    'Business': ['Business trip', 'Work trip', 'Conference', 'Business'],
    'Romantic': ['Romantic trip', 'Honeymoon', 'Anniversary', 'Romantic'],
    'Family': ['Family trip', 'With children', 'Family'],
    'Solo': ['Solo trip', 'Alone', 'Solo']
}
# Функция извлечения тип поездки
def extract_trip_type(tags_str):
    tags = str(tags_str).lower()
    if re.search(r'leisure|vacation|holiday', tags):
        return 'Leisure'
    if re.search(r'business|work|conference', tags):
        return 'Business'
    if re.search(r'romantic|honeymoon|anniversary', tags):
        return 'Romantic'
    return 'Other'
# Применяем функцию
data['trip_type'] = data['tags'].apply(extract_trip_type)

In [55]:
data['trip_type'].value_counts()

trip_type
Leisure     313602
Business     61998
Other        11202
Romantic         1
Name: count, dtype: int64

In [57]:
def extract_guests_type(tags_str):
    # Cловарь типов гостей с вариантами написания
    guests_types = {
        'Solo': ['Solo traveler', 'Alone', 'Solo', 'Single', 'Traveled alone', 'Business trip'],
        'Couple': ['Couple', 'Two adults', 'Duo', 'Pair', 'Romantic trip', 'Honeymoon'],
        'Family': ['Family', 'With children', 'Kids', 'Child', 'Parents', 'Family trip'],
        'Group': ['Group', 'Friends', 'Colleagues', 'Team', 'Several people', 'Many travelers'],
        'Business': ['Colleague', 'Coworkers', 'Business partners', 'Conference attendees']
    }
    # Нормализуем строку тегов
    tags = str(tags_str).lower()
    # Ищем совпадения
    for guest_type, keywords in guests_types.items():
        for keyword in keywords:
            if keyword.lower() in tags:
                return guest_type  
    # Дополнительные проверки по контексту
    if 'child' in tags or 'kid' in tags:
        return 'Family'
    if 'friend' in tags:
        return 'Group'
    return 'Unknown'  # Вместо None для сохранения всех записей
# Применение
data['guests_type'] = data['tags'].apply(extract_guests_type)

In [58]:
data['guests_type'].value_counts()

guests_type
Couple    177795
Solo      102668
Family     63161
Group      43179
Name: count, dtype: int64

In [ ]:
# Функция для извлечения категории номера
def extract_category_room(tags_str):
    tags = [t.strip().strip("'") for t in tags_str.strip("[]").split(",")]
    trip_types = ['Standard Double', 'Studio Suite', 'Deluxe Room']
    for tag in tags:
        if tag in trip_types:
            return tag
    return None  # если тип номера не найден

# Применяем функцию к столбцу с тегами
data['category_room'] = data['tags'].apply(extract_category_room)

In [ ]:
# Функция для извлечения длительности пребывания
def extract_timeofstay(tags_str):
    tags = [t.strip().strip("'") for t in tags_str.strip("[]").split(",")]
    trip_types = ['Stayed 1 night', 'Stayed 2 nights', 'Long stay']
    for tag in tags:
        if tag in trip_types:
            return tag
    return None  # если тип времяпребывания не найден

# Применяем функцию к столбцу с тегами
data['timeofstay'] = data['tags'].apply(extract_timeofstay)

In [ ]:
# Удаление старого признака
data = data.drop(['tags'], axis=1)

### 1.8 Дни с даты ревью

In [ ]:
data['days_since_review'].head(2)

Можно разделить на группы, насколько давно был оставлен отзыв

In [ ]:
data['days_since_review'].apply(lambda x: x.split()[1]).value_counts()

Содержит только дни, значит можно без перевода из годов в дни получить дни

In [ ]:
data['days_since_review'] = data['days_since_review'].apply(lambda x: int(x.split()[0]))
bins = [0, 30, 90, 365, float('inf')]  # Границы категорий (дни)
labels = ['<1 month', '1-3 months', '3-12 months', '>1 year']
data['review_recency'] = pd.cut(data['days_since_review'], bins=bins, labels=labels)

In [ ]:
# Удаление старого признака
data = data.drop(['days_since_review'], axis=1)

## 2. Работа с пропусками в данных

In [ ]:
# Вывод общей информации о столбцах
data.info()

In [ ]:
# Получаем список столбцов с пропусками
columns_with_nulls = data.columns[data.isnull().any()].tolist()
# Выводим результат
print("Столбцы с пропущенными значениями:")
print(columns_with_nulls)

### 2.1 Географическая широта

In [ ]:
data['lat'].head(2)

In [ ]:
# Заполнение медианой в пределах города
data['lat'] = data.groupby('city')['lat'].transform(lambda x: x.fillna(x.median()))

Если модель будет плохо сходится - можно попробовать заполнять в пределах региона

### 2.2 Бренд

Поскольку этот признак вводился на этапе обработки, огично заполнить то, что не категоризировалось, значением "другие"

In [ ]:
# Находим самый частый бренд (моду)
most_common_brand = data['brand'].mode()[0]
# Заменяем пропуски на этот бренд
data['brand'] = data['brand'].fillna(most_common_brand)

In [ ]:
data['brand'].value_counts()

### 2.3 Тип заведения

In [ ]:
data['type'] = data['type'].fillna('Other')

In [ ]:
data['type'].value_counts()

### 2.4 Тип поездки

In [ ]:
data['trip_type']

Видимо я опрометчиво выбрала категории. Надо посмотреть на исходный

### 2.5 Тип гостей

In [ ]:
data['guests_type'].value_counts()

### 2.6 Категория комнаты

In [ ]:
data['category_room'].value_counts()

### 2.7 Время остановки

In [ ]:
data['timeofstay'].value_counts()

## 3. Создание новых признаков

## 4. Преборазование признаков

## 5. Отбор признаков